In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import balanced_accuracy_score, f1_score, confusion_matrix, matthews_corrcoef
import sys
sys.path.append('/home/ss2686/03_DICTrank')
import argparse
from scripts.evaluation_functions import evaluate_classifier, optimize_threshold_j_statistic

In [2]:
str_results = pd.read_csv("../run_all_models_scripts/structural_model_held_out_test_results.csv")
phy_results = pd.read_csv("../run_all_models_scripts/physicochemical_model_held_out_test_results.csv")
img_results = pd.read_csv("../run_all_models_scripts/cellpainting_model_held_out_test_results.csv")
ge_results = pd.read_csv("../run_all_models_scripts/LINCSL1000_model_held_out_test_results.csv")
go_results = pd.read_csv("../run_all_models_scripts/GO_model_held_out_test_results.csv")
pid_results = pd.read_csv("../run_all_models_scripts/CellScape_model_held_out_test_results.csv") 
pid_cmu_results = pd.read_csv("../run_all_models_scripts/CellScape_Cmax_unbound_model_held_out_test_results.csv") 
pid_cmt_results = pd.read_csv("../run_all_models_scripts/CellScape_Cmax_total_model_held_out_test_results.csv") 
moa_results = pd.read_csv("../run_all_models_scripts/MOA_model_held_out_test_results.csv") 
moa_cmu_results = pd.read_csv("../run_all_models_scripts/MOA_Cmax_unbound_model_held_out_test_results.csv") 
moa_cmt_results = pd.read_csv("../run_all_models_scripts/MOA_Cmax_total_model_held_out_test_results.csv") 

str_results["Features"] ="Structural"
phy_results["Features"] ="Mordred"
img_results["Features"] ="Cell Painting"
ge_results["Features"] ="LINCSL1000"
go_results["Features"] ="GeneOntology"
pid_results["Features"] ="CellScape"
pid_cmu_results["Features"] ="CellScape with Cmax unbound"
pid_cmt_results["Features"] ="CellScape with Cmax total"
moa_results["Features"] ="MOA"
moa_cmu_results["Features"] ="MOA with Cmax unbound"
moa_cmt_results["Features"] ="MOA with Cmax total"

In [3]:
str_results

,Dataset,Actviity,SMILES,True_Value,Prediction,Probability,Best_Threshold,Features
0,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),O=C(C1CCCCC1)[NH+]1CC(=O)[NH+]2CCc3ccccc3C2C1,1,1,0.664392,0.541602,Structural
1,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),CC(C(=O)[O-])c1ccc(-c2ccccc2)c(F)c1,1,1,0.562749,0.541602,Structural
2,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),C[NH+](C)CCC=C1c2ccccc2CCc2ccccc21,1,1,0.613492,0.541602,Structural
3,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),CC(=O)[NH+]1CCN(c2ccc(OCC3COC(Cn4ccnc4)(c4ccc(...,1,0,0.441763,0.541602,Structural
4,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),Cc1nccn1CC1CCc2c(c3ccccc3n2C)C1=O,1,1,0.551398,0.541602,Structural
...,...,...,...,...,...,...,...,...
488,DICTrank,DICTrank,CCC1(c2ccccc2)C(=O)NCNC1=O,0,0,0.536495,0.549585,Structural
489,DICTrank,DICTrank,CCOC(=O)[NH+]1CCC(=C2c3ccc(Cl)cc3CCc3cccnc32)CC1,0,1,0.555763,0.549585,Structural
490,DICTrank,DICTrank,CCCSc1ccc2[n-]c(=NC(=O)OC)[n-]c2c1,0,0,0.523943,0.549585,Structural
491,DICTrank,DICTrank,CCC[NH+](CCC)S(=O)(=O)c1ccc(C(=O)[O-])cc1,0,1,0.611275,0.549585,Structural


In [4]:
allresults = pd.concat([str_results, phy_results, img_results, ge_results, go_results, 
                        pid_results, 
                        moa_results, pid_cmu_results,
                        pid_cmt_results, moa_cmu_results,
                        moa_cmt_results
                       ]).sort_values(["Dataset", "Features", "SMILES"],
                                                  ascending=[False, False, False]).reset_index(drop=True)
allresults

,Dataset,Actviity,SMILES,True_Value,Prediction,Probability,Best_Threshold,Features
0,sider_cardiacdisorders,Cardiac disorders,c1ccc2c(CC3=[NH+]CCN3)cccc2c1,0,1,0.574822,0.516513,Structural
1,sider_cardiacdisorders,Cardiac disorders,[O-]c1ccc(C2C[NH2+]CCc3c2cc([O-])c([O-])c3Cl)cc1,1,1,0.535648,0.516513,Structural
2,sider_cardiacdisorders,Cardiac disorders,[NH3+]C(Cc1cc(I)c(Oc2cc(I)c([O-])c(I)c2)c(I)c1...,1,0,0.490015,0.516513,Structural
3,sider_cardiacdisorders,Cardiac disorders,OCC[NH+]1CC[NH+](CCCN2c3ccccc3Sc3ccc(C(F)(F)F)...,1,1,0.741655,0.516513,Structural
4,sider_cardiacdisorders,Cardiac disorders,O=c1n(CCC[NH+]2CCN(c3cccc(Cl)c3)CC2)nc2ccccn12,1,1,0.638864,0.516513,Structural
...,...,...,...,...,...,...,...,...
4760,DICTrank,DICTrank,CC(=O)N=c1[n-]nc(S(N)(=O)=O)s1,0,1,0.832073,0.701560,Cell Painting
4761,DICTrank,DICTrank,CC(=O)CC(c1ccccc1)c1c([O-])c2ccccc2oc1=O,0,1,0.911894,0.701560,Cell Painting
4762,DICTrank,DICTrank,C=CC[NH+]1CCC23c4c5ccc([O-])c4OC2C(=O)CCC3(O)C1C5,1,1,0.889940,0.701560,Cell Painting
4763,DICTrank,DICTrank,C=CC1=C(C(=O)[O-])[NH+]2C(=O)C(=NC(=O)C([NH2+]...,0,1,0.807566,0.701560,Cell Painting


In [5]:
allresults[allresults["Dataset"]=="DICTrank"].Features.value_counts()

Structural                     90
Mordred                        90
MOA with Cmax total            90
MOA                            90
LINCSL1000                     90
GeneOntology                   90
CellScape with Cmax total      90
CellScape                      90
Cell Painting                  90
MOA with Cmax unbound          78
CellScape with Cmax unbound    78
Name: Features, dtype: int64

In [6]:
def scale_probability(prob, threshold):
    if prob <= threshold:
        return prob / (2 * threshold)
    else:
        return 1 - (1 - prob) / (2 * (1 - threshold))

allresults['Scaled_Probability'] = allresults.apply(lambda row: scale_probability(row['Probability'], row['Best_Threshold']), axis=1)
allresults

,Dataset,Actviity,SMILES,True_Value,Prediction,Probability,Best_Threshold,Features,Scaled_Probability
0,sider_cardiacdisorders,Cardiac disorders,c1ccc2c(CC3=[NH+]CCN3)cccc2c1,0,1,0.574822,0.516513,Structural,0.560300
1,sider_cardiacdisorders,Cardiac disorders,[O-]c1ccc(C2C[NH2+]CCc3c2cc([O-])c([O-])c3Cl)cc1,1,1,0.535648,0.516513,Structural,0.519788
2,sider_cardiacdisorders,Cardiac disorders,[NH3+]C(Cc1cc(I)c(Oc2cc(I)c([O-])c(I)c2)c(I)c1...,1,0,0.490015,0.516513,Structural,0.474349
3,sider_cardiacdisorders,Cardiac disorders,OCC[NH+]1CC[NH+](CCCN2c3ccccc3Sc3ccc(C(F)(F)F)...,1,1,0.741655,0.516513,Structural,0.732831
4,sider_cardiacdisorders,Cardiac disorders,O=c1n(CCC[NH+]2CCN(c3cccc(Cl)c3)CC2)nc2ccccn12,1,1,0.638864,0.516513,Structural,0.626530
...,...,...,...,...,...,...,...,...,...
4760,DICTrank,DICTrank,CC(=O)N=c1[n-]nc(S(N)(=O)=O)s1,0,1,0.832073,0.701560,Cell Painting,0.718659
4761,DICTrank,DICTrank,CC(=O)CC(c1ccccc1)c1c([O-])c2ccccc2oc1=O,0,1,0.911894,0.701560,Cell Painting,0.852388
4762,DICTrank,DICTrank,C=CC[NH+]1CCC23c4c5ccc([O-])c4OC2C(=O)CCC3(O)C1C5,1,1,0.889940,0.701560,Cell Painting,0.815608
4763,DICTrank,DICTrank,C=CC1=C(C(=O)[O-])[NH+]2C(=O)C(=NC(=O)C([NH2+]...,0,1,0.807566,0.701560,Cell Painting,0.677600


In [7]:
allresults.Actviity.unique()

array(['Cardiac disorders', 'Cardiotox (with SIDER inactives)',
       'Cardiotox (with SIDER all)', 'Cardiotox (with SIDER actives)',
       'DICTrank'], dtype=object)

In [8]:
for dataset, activity in [('sider_cardiacdisorders','Cardiac disorders'), 
                           ('cardiotox_with_sider_inactives','Cardiotox (with SIDER inactives)'),
           ('cardiotox_with_sider_all','Cardiotox (with SIDER all)'), 
                           ('cardiotox_with_sider_actives','Cardiotox (with SIDER actives)'),
           ('DICTrank', 'DICTrank')]:
            
            print(activity)
            print(dataset)

Cardiac disorders
sider_cardiacdisorders
Cardiotox (with SIDER inactives)
cardiotox_with_sider_inactives
Cardiotox (with SIDER all)
cardiotox_with_sider_all
Cardiotox (with SIDER actives)
cardiotox_with_sider_actives
DICTrank
DICTrank


In [9]:
allresults

,Dataset,Actviity,SMILES,True_Value,Prediction,Probability,Best_Threshold,Features,Scaled_Probability
0,sider_cardiacdisorders,Cardiac disorders,c1ccc2c(CC3=[NH+]CCN3)cccc2c1,0,1,0.574822,0.516513,Structural,0.560300
1,sider_cardiacdisorders,Cardiac disorders,[O-]c1ccc(C2C[NH2+]CCc3c2cc([O-])c([O-])c3Cl)cc1,1,1,0.535648,0.516513,Structural,0.519788
2,sider_cardiacdisorders,Cardiac disorders,[NH3+]C(Cc1cc(I)c(Oc2cc(I)c([O-])c(I)c2)c(I)c1...,1,0,0.490015,0.516513,Structural,0.474349
3,sider_cardiacdisorders,Cardiac disorders,OCC[NH+]1CC[NH+](CCCN2c3ccccc3Sc3ccc(C(F)(F)F)...,1,1,0.741655,0.516513,Structural,0.732831
4,sider_cardiacdisorders,Cardiac disorders,O=c1n(CCC[NH+]2CCN(c3cccc(Cl)c3)CC2)nc2ccccn12,1,1,0.638864,0.516513,Structural,0.626530
...,...,...,...,...,...,...,...,...,...
4760,DICTrank,DICTrank,CC(=O)N=c1[n-]nc(S(N)(=O)=O)s1,0,1,0.832073,0.701560,Cell Painting,0.718659
4761,DICTrank,DICTrank,CC(=O)CC(c1ccccc1)c1c([O-])c2ccccc2oc1=O,0,1,0.911894,0.701560,Cell Painting,0.852388
4762,DICTrank,DICTrank,C=CC[NH+]1CCC23c4c5ccc([O-])c4OC2C(=O)CCC3(O)C1C5,1,1,0.889940,0.701560,Cell Painting,0.815608
4763,DICTrank,DICTrank,C=CC1=C(C(=O)[O-])[NH+]2C(=O)C(=NC(=O)C([NH2+]...,0,1,0.807566,0.701560,Cell Painting,0.677600


In [10]:
allresults.groupby("Dataset").Features.value_counts()

Dataset                         Features                   
DICTrank                        Cell Painting                   90
                                CellScape                       90
                                CellScape with Cmax total       90
                                GeneOntology                    90
                                LINCSL1000                      90
                                MOA                             90
                                MOA with Cmax total             90
                                Mordred                         90
                                Structural                      90
                                CellScape with Cmax unbound     78
                                MOA with Cmax unbound           78
cardiotox_with_sider_actives    Cell Painting                   90
                                CellScape                       90
                                CellScape with Cmax total       90
  

In [11]:
smaller_test = (moa_cmu_results[moa_cmu_results["Dataset"]=="DICTrank"].SMILES.to_list())
len(smaller_test)

78

In [12]:
# Create an empty list to store DataFrames
ensemble_predictions_dfs = []
results = {}

# Loop through each dataset
for dataset, activity in [#('sider_cardiacdisorders','Cardiac disorders'), 
                          ('cardiotox_with_sider_inactives','Cardiotox (with SIDER inactives)'),
                          ('cardiotox_with_sider_all','Cardiotox (with SIDER all)'), 
                          ('cardiotox_with_sider_actives','Cardiotox (with SIDER actives)'),
                          ('DICTrank', 'DICTrank')]:
        print(dataset)
        df_dataset = allresults[allresults["Dataset"] == dataset]
        df_dataset = df_dataset[df_dataset.SMILES.isin(smaller_test)].reset_index(drop=True)
        
        df_ensemble_all = df_dataset.groupby("SMILES").mean().reset_index()
        ensemble_probabilities = df_ensemble_all["Scaled_Probability"]

        # Convert ensemble_proba_dict to ensemble predictions based on threshold
        ensemble_predictions = df_ensemble_all["Scaled_Probability"].apply(lambda x: 1 if x > 0.5 else 0)
        
        
        results[activity] = {
                **evaluate_classifier(df_ensemble_all["True_Value"].values, ensemble_predictions, ensemble_probabilities)
            }

        ensemble_predictions_df = pd.DataFrame()
        
        ensemble_predictions_df["SMILES"] = df_ensemble_all["SMILES"]
        ensemble_predictions_df['Dataset'] = dataset
        ensemble_predictions_df['Actviity'] = activity
        ensemble_predictions_df['True_Value'] = df_ensemble_all["True_Value"]
        ensemble_predictions_df['Prediction'] = ensemble_predictions
        ensemble_predictions_df['Probability'] = ensemble_probabilities
        ensemble_predictions_df['Best_Threshold'] = 0.50
        ensemble_predictions_df['Features'] = "Ensemble_all"
        
        ensemble_predictions_dfs.append(ensemble_predictions_df)

# Concatenate all DataFrames into one unified DataFrame
combined_df = pd.concat(ensemble_predictions_dfs, ignore_index=True)        
combined_df.to_csv(f"Ensemble_all_models_held_out_test_results.csv", index=False)

# Save results
results_df = pd.DataFrame(results).T.reset_index(drop=False)
results_df = results_df.rename(columns={'index': 'endpoint'})
results_df.to_csv('./Ensemble_all_model_results.csv', index=False)

cardiotox_with_sider_inactives
cardiotox_with_sider_all
cardiotox_with_sider_actives
DICTrank


In [13]:
combined_df 

,SMILES,Dataset,Actviity,True_Value,Prediction,Probability,Best_Threshold,Features
0,C=CC1=C(C(=O)[O-])[NH+]2C(=O)C(=NC(=O)C([NH2+]...,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),0.0,0,0.455237,0.5,Ensemble_all
1,C=CC[NH+]1CCC23c4c5ccc([O-])c4OC2C(=O)CCC3(O)C1C5,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),1.0,1,0.645761,0.5,Ensemble_all
2,CC(=O)CC(c1ccccc1)c1c([O-])c2ccccc2oc1=O,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),0.0,0,0.480446,0.5,Ensemble_all
3,CC(=O)N=c1[n-]nc(S(N)(=O)=O)s1,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),0.0,0,0.434869,0.5,Ensemble_all
4,CC(=O)[NH+]1CCN(c2ccc(OCC3COC(Cn4ccnc4)(c4ccc(...,cardiotox_with_sider_inactives,Cardiotox (with SIDER inactives),1.0,0,0.472898,0.5,Ensemble_all
...,...,...,...,...,...,...,...,...
307,O=C1[N-]C(=O)C(c2ccccc2)(c2ccccc2)N1,DICTrank,DICTrank,1.0,1,0.525787,0.5,Ensemble_all
308,O=c1n(CCC[NH+]2CCN(c3cccc(Cl)c3)CC2)nc2ccccn12,DICTrank,DICTrank,1.0,1,0.706334,0.5,Ensemble_all
309,OC(Cn1cncn1)(Cn1cncn1)c1ccc(F)cc1F,DICTrank,DICTrank,1.0,1,0.514773,0.5,Ensemble_all
310,OC(c1cc(C(F)(F)F)nc2c(C(F)(F)F)cccc12)C1CCCC[N...,DICTrank,DICTrank,1.0,1,0.658556,0.5,Ensemble_all


In [14]:
combined_df.Dataset.value_counts()

cardiotox_with_sider_inactives    78
cardiotox_with_sider_all          78
cardiotox_with_sider_actives      78
DICTrank                          78
Name: Dataset, dtype: int64

In [15]:
combined_df.groupby("Dataset").Features.value_counts()

Dataset                         Features    
DICTrank                        Ensemble_all    78
cardiotox_with_sider_actives    Ensemble_all    78
cardiotox_with_sider_all        Ensemble_all    78
cardiotox_with_sider_inactives  Ensemble_all    78
Name: Features, dtype: int64

In [16]:
results_df

,endpoint,Held_out_TP,Held_out_TN,Held_out_FP,Held_out_FN,Held_out_BA,Held_out_AUC,Held_out_MCC,Held_out_AUCPR,Held_out_Specificity,Held_out_Sensitivity,Held_out_F1
0,Cardiotox (with SIDER inactives),46.0,13.0,8.0,11.0,0.713033,0.805347,0.409469,0.919904,0.619048,0.807018,0.828829
1,Cardiotox (with SIDER all),45.0,13.0,8.0,12.0,0.704261,0.832080,0.388288,0.933899,0.619048,0.789474,0.818182
2,Cardiotox (with SIDER actives),47.0,12.0,9.0,10.0,0.697995,0.825397,0.390325,0.930765,0.571429,0.824561,0.831858
3,DICTrank,51.0,10.0,11.0,6.0,0.685464,0.830409,0.407455,0.934383,0.476190,0.894737,0.857143


In [17]:
df_dataset[df_dataset["SMILES"]=="OCCOCC[NH+]1CC[NH+](C(c2ccccc2)c2ccc(Cl)cc2)CC1"].Scaled_Probability.mean()

0.6234365291765085

In [18]:
df_dataset

,Dataset,Actviity,SMILES,True_Value,Prediction,Probability,Best_Threshold,Features,Scaled_Probability
0,DICTrank,DICTrank,OCCOCC[NH+]1CC[NH+](C(c2ccccc2)c2ccc(Cl)cc2)CC1,1,1,0.638441,0.549585,Structural,0.598638
1,DICTrank,DICTrank,OC(c1cc(C(F)(F)F)nc2c(C(F)(F)F)cccc12)C1CCCC[N...,1,1,0.653451,0.549585,Structural,0.615300
2,DICTrank,DICTrank,OC(Cn1cncn1)(Cn1cncn1)c1ccc(F)cc1F,1,0,0.538468,0.549585,Structural,0.489886
3,DICTrank,DICTrank,O=c1n(CCC[NH+]2CCN(c3cccc(Cl)c3)CC2)nc2ccccn12,1,1,0.619788,0.549585,Structural,0.577932
4,DICTrank,DICTrank,O=C1[N-]C(=O)C(c2ccccc2)(c2ccccc2)N1,1,0,0.539547,0.549585,Structural,0.490868
...,...,...,...,...,...,...,...,...,...
853,DICTrank,DICTrank,CC(=O)[NH+]1CCN(c2ccc(OCC3COC(Cn4ccnc4)(c4ccc(...,1,1,0.852371,0.701560,Cell Painting,0.752665
854,DICTrank,DICTrank,CC(=O)N=c1[n-]nc(S(N)(=O)=O)s1,0,1,0.832073,0.701560,Cell Painting,0.718659
855,DICTrank,DICTrank,CC(=O)CC(c1ccccc1)c1c([O-])c2ccccc2oc1=O,0,1,0.911894,0.701560,Cell Painting,0.852388
856,DICTrank,DICTrank,C=CC[NH+]1CCC23c4c5ccc([O-])c4OC2C(=O)CCC3(O)C1C5,1,1,0.889940,0.701560,Cell Painting,0.815608


In [19]:
df_dataset.Features.unique()

array(['Structural', 'Mordred', 'MOA with Cmax unbound',
       'MOA with Cmax total', 'MOA', 'LINCSL1000', 'GeneOntology',
       'CellScape with Cmax unbound', 'CellScape with Cmax total',
       'CellScape', 'Cell Painting'], dtype=object)

In [20]:
larger_test_selected_list= ['Structural', 'Mordred', 'MOA', 'CellScape',
                           'CellScape with Cmax total', 'MOA with Cmax total']

In [21]:
# Create an empty list to store DataFrames
ensemble_predictions_dfs = []
results = {}

# Loop through each dataset
for dataset, activity in [('sider_cardiacdisorders','Cardiac disorders'), 
                          ('cardiotox_with_sider_inactives','Cardiotox (with SIDER inactives)'),
                          ('cardiotox_with_sider_all','Cardiotox (with SIDER all)'), 
                          ('cardiotox_with_sider_actives','Cardiotox (with SIDER actives)'),
                          ('DICTrank', 'DICTrank')]:
        print(dataset)
        df_dataset = allresults[allresults["Dataset"] == dataset]
        # Ensemble_ALL
        grouped = df_dataset.groupby('SMILES')
        # Collect ensemble predictions
        ensemble_proba_dict = {}

        df_ensemble_all = df_dataset[df_dataset.Features.isin(larger_test_selected_list)].groupby("SMILES").mean().reset_index()
        ensemble_probabilities = df_ensemble_all["Scaled_Probability"]

        # Convert ensemble_proba_dict to ensemble predictions based on threshold
        ensemble_predictions = df_ensemble_all["Scaled_Probability"].apply(lambda x: 1 if x > 0.5 else 0)
        
        
        results[activity] = {
                **evaluate_classifier(df_ensemble_all["True_Value"].values, ensemble_predictions, ensemble_probabilities)
            }

        ensemble_predictions_df = pd.DataFrame()
        
        ensemble_predictions_df["SMILES"] = df_ensemble_all["SMILES"]
        ensemble_predictions_df['Dataset'] = dataset
        ensemble_predictions_df['Actviity'] = activity
        ensemble_predictions_df['True_Value'] = df_ensemble_all["True_Value"]
        ensemble_predictions_df['Prediction'] = ensemble_predictions
        ensemble_predictions_df['Probability'] = ensemble_probabilities
        ensemble_predictions_df['Best_Threshold'] = 0.50
        ensemble_predictions_df['Features'] = "Ensemble_selected"
        
        ensemble_predictions_dfs.append(ensemble_predictions_df)

# Concatenate all DataFrames into one unified DataFrame
combined_df = pd.concat(ensemble_predictions_dfs, ignore_index=True)        
combined_df.to_csv(f"Ensemble_selected_models_held_out_test_results.csv", index=False)

# Save results
results_df = pd.DataFrame(results).T.reset_index(drop=False)
results_df = results_df.rename(columns={'index': 'endpoint'})
results_df.to_csv('./Ensemble_selected_model_results.csv', index=False)

sider_cardiacdisorders
cardiotox_with_sider_inactives
cardiotox_with_sider_all
cardiotox_with_sider_actives
DICTrank


In [22]:
combined_df

,SMILES,Dataset,Actviity,True_Value,Prediction,Probability,Best_Threshold,Features
0,C#CC1(O)CCC2C3CCC4=CCCCC4C3C(=C)CC21CC,sider_cardiacdisorders,Cardiac disorders,0.0,0,0.444618,0.5,Ensemble_selected
1,C=CC1=C(C)c2cc3[nH]c(cc4[nH]c(cc5nc(cc1n2)C(C)...,sider_cardiacdisorders,Cardiac disorders,0.0,1,0.607163,0.5,Ensemble_selected
2,CC(=CC(=O)OCCCCCCCCC(=O)[O-])CC1OCC(CC2OC2C(C)...,sider_cardiacdisorders,Cardiac disorders,0.0,1,0.542064,0.5,Ensemble_selected
3,CC(=O)C(C#N)C(=O)Nc1ccc(C(F)(F)F)cc1,sider_cardiacdisorders,Cardiac disorders,1.0,1,0.512995,0.5,Ensemble_selected
4,CC(=O)C(O)O,sider_cardiacdisorders,Cardiac disorders,0.0,0,0.411872,0.5,Ensemble_selected
...,...,...,...,...,...,...,...,...
488,O=C1[N-]C(=O)C(c2ccccc2)(c2ccccc2)N1,DICTrank,DICTrank,1.0,1,0.522085,0.5,Ensemble_selected
489,O=c1n(CCC[NH+]2CCN(c3cccc(Cl)c3)CC2)nc2ccccn12,DICTrank,DICTrank,1.0,1,0.765879,0.5,Ensemble_selected
490,OC(Cn1cncn1)(Cn1cncn1)c1ccc(F)cc1F,DICTrank,DICTrank,1.0,1,0.526656,0.5,Ensemble_selected
491,OC(c1cc(C(F)(F)F)nc2c(C(F)(F)F)cccc12)C1CCCC[N...,DICTrank,DICTrank,1.0,1,0.619225,0.5,Ensemble_selected


In [23]:
combined_df.groupby("Dataset").Features.value_counts()

Dataset                         Features         
DICTrank                        Ensemble_selected     90
cardiotox_with_sider_actives    Ensemble_selected     90
cardiotox_with_sider_all        Ensemble_selected     90
cardiotox_with_sider_inactives  Ensemble_selected     90
sider_cardiacdisorders          Ensemble_selected    133
Name: Features, dtype: int64

In [24]:
results_df

,endpoint,Held_out_TP,Held_out_TN,Held_out_FP,Held_out_FN,Held_out_BA,Held_out_AUC,Held_out_MCC,Held_out_AUCPR,Held_out_Specificity,Held_out_Sensitivity,Held_out_F1
0,Cardiac disorders,69.0,23.0,17.0,24.0,0.658468,0.718817,0.304049,0.863481,0.575,0.741935,0.770950
1,Cardiotox (with SIDER inactives),46.0,19.0,6.0,19.0,0.733846,0.788308,0.424125,0.909336,0.760,0.707692,0.786325
2,Cardiotox (with SIDER all),41.0,21.0,4.0,24.0,0.735385,0.791385,0.421718,0.912928,0.840,0.630769,0.745455
3,Cardiotox (with SIDER actives),55.0,11.0,14.0,10.0,0.643077,0.800615,0.303035,0.915569,0.440,0.846154,0.820896
4,DICTrank,56.0,13.0,12.0,9.0,0.690769,0.827692,0.397648,0.925051,0.520,0.861538,0.842105


In [25]:
k =df_dataset[df_dataset.Features.isin(larger_test_selected_list)]
k[k["SMILES"]=="OCCOCC[NH+]1CC[NH+](C(c2ccccc2)c2ccc(Cl)cc2)CC1"].Scaled_Probability.mean()

0.6418678468490081

In [26]:
df_dataset

,Dataset,Actviity,SMILES,True_Value,Prediction,Probability,Best_Threshold,Features,Scaled_Probability
3799,DICTrank,DICTrank,OCCOCC[NH+]1CC[NH+](C(c2ccccc2)c2ccc(Cl)cc2)CC1,1,1,0.638441,0.549585,Structural,0.598638
3800,DICTrank,DICTrank,OC(c1cc(C(F)(F)F)nc2c(C(F)(F)F)cccc12)C1CCCC[N...,1,1,0.653451,0.549585,Structural,0.615300
3801,DICTrank,DICTrank,OC(Cn1cncn1)(Cn1cncn1)c1ccc(F)cc1F,1,0,0.538468,0.549585,Structural,0.489886
3802,DICTrank,DICTrank,O=c1n(CCC[NH+]2CCN(c3cccc(Cl)c3)CC2)nc2ccccn12,1,1,0.619788,0.549585,Structural,0.577932
3803,DICTrank,DICTrank,O=C1[N-]C(=O)C(c2ccccc2)(c2ccccc2)N1,1,0,0.539547,0.549585,Structural,0.490868
...,...,...,...,...,...,...,...,...,...
4760,DICTrank,DICTrank,CC(=O)N=c1[n-]nc(S(N)(=O)=O)s1,0,1,0.832073,0.701560,Cell Painting,0.718659
4761,DICTrank,DICTrank,CC(=O)CC(c1ccccc1)c1c([O-])c2ccccc2oc1=O,0,1,0.911894,0.701560,Cell Painting,0.852388
4762,DICTrank,DICTrank,C=CC[NH+]1CCC23c4c5ccc([O-])c4OC2C(=O)CCC3(O)C1C5,1,1,0.889940,0.701560,Cell Painting,0.815608
4763,DICTrank,DICTrank,C=CC1=C(C(=O)[O-])[NH+]2C(=O)C(=NC(=O)C([NH2+]...,0,1,0.807566,0.701560,Cell Painting,0.677600
